In [49]:
# pip install requests
# pip install beautifulsoup4
# pip install nltk 
# pip install spacy
# pip install textblob
# pip install wordcloud

In [50]:
import requests 
from bs4 import BeautifulSoup
import os

In [51]:
# URL des Wahlprogramms abrufen

url = 'https://www.fdp.de/das-wahlprogramm-der-freien-demokraten-zur-bundestagswahl-2025'

response  = requests.get(url)
if response.status_code == 200:
     page = response.text
else:
    print(f"Seite konnte nicht abgerufen werden - Status Code {response.status_code}")

soup = BeautifulSoup(page, 'html.parser')

del page, response

In [53]:
# Liste zum Speichern der Inhalte in der richtigen Reihenfolge
content_list = []

# Erstes <h2>-Element suchen, um den Startpunkt zu setzen
start_element = soup.find("h2", class_="subheader")

# Falls ein <h2> gefunden wurde, ab diesem Element iterieren
if start_element:
    # Erstes h2-Element explizit hinzufügen
    content_list.append(f"### {start_element.get_text(strip=True)}")

    under_h3 = False  # Variable, um zu verfolgen, ob wir uns unter einer h3-Überschrift befinden
    last_h2_index = None  # Index des letzten gespeicherten h2
    downgrade_h2 = False  # Ob das letzte h2 zu ## umgewandelt werden muss

    # Alle nachfolgenden relevanten Elemente sammeln
    for element in start_element.find_all_next(["h2", "h3", "label", "p"]):
        # Überschriften mit ### markieren
        if element.name == "h2" and "subheader" in element.get("class", []):
            text = element.get_text(strip=True)
            content_list.append(f"### {text}")
            last_h2_index = len(content_list) - 1  # Speichert den Index des letzten h2
            downgrade_h2 = False  # Standardmäßig bleibt es ###

        # h3-Überschriften erkennen
        elif element.name == "h3":
            under_h3 = True  # Ab jetzt sind wir unter einer h3-Überschrift

        # Thesen mit ## markieren, aber "Unsere Mission" ignorieren
        elif element.name == "label" and "accordion-label" in element.get("class", []):
            text = element.get_text(strip=True)
            if text == "Unsere Mission":
                downgrade_h2 = True  # Falls vorher ein h2 war, soll es zu ## geändert werden
            else:
                content_list.append(f"## {text}")
                under_h3 = False  # Falls eine These kommt, sind wir nicht mehr unter einer h3

        # Falls das vorherige h2 downgegradet werden soll
        if downgrade_h2 and last_h2_index is not None:
            content_list[last_h2_index] = content_list[last_h2_index].replace("###", "##")
            downgrade_h2 = False  # Rücksetzen, damit es nur einmal passiert

        # Alle Texte extrahieren, wenn sie NICHT unter einer h3 stehen
        elif element.name == "p" and not under_h3:
            text = element.get_text(" ", strip=True)  # Leerzeichen für bessere Lesbarkeit
            content_list.append(f"# {text}")

# Entferne komplett leere Einträge oder Einträge mit nur Leerzeichen
content_list = [entry.strip() for entry in content_list if entry.strip()]

# Die extrahierten Inhalte speichern
fdp_gesamt = "\n".join(content_list)
with open("fdp.txt", "w", encoding="utf-8") as file:
    file.write(fdp_gesamt)

# Gesamtwortanzahl berechnen
word_count = len(fdp_gesamt.split())
print(f"\nGesamtanzahl der Wörter: {word_count}")


Gesamtanzahl der Wörter: 19237
